In [1]:
import torch as th
import torch
import torch.nn as nn
import time
import math
import copy

/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class TCNN(th.nn.Module):
    def __init__(self, seq_len, channels, ntoken) -> None:
        super().__init__()
        self.model = th.nn.Sequential()
        self.encoder = nn.Embedding(ntoken, channels[0])
        self.decoder = nn.Linear(channels[-1], ntoken)
        for i in range(len(channels)-1):
            self.model.append(th.nn.Conv1d(channels[i], channels[i+1], kernel_size=seq_len, stride=1, padding='same'))
            if i < len(channels) - 2:
                self.model.append(th.nn.BatchNorm1d(num_features=channels[i+1]))
                self.model.append(th.nn.ReLU())
    
    def forward(self, input:th.Tensor):
        embedded = self.encoder.forward(input)
        embedded = embedded.permute(1,2,0)
        internal_result = self.model.forward(embedded)
        internal_result = internal_result.permute(2,0,1)
        result = self.decoder(internal_result)
        return result

In [3]:
class TMLP(th.nn.Module):
    def __init__(self, seq_len, arch, ntoken) -> None:
        super().__init__()
        self.model = th.nn.Sequential()
        self.encoder = nn.Embedding(ntoken, arch[0])
        self.decoder = nn.Linear(arch[-1], ntoken)
        arch[0] *= seq_len
        arch[-1]*= seq_len
        for i in range(len(arch)-1):
            self.model.append(th.nn.Linear(arch[i], arch[i+1]))
            if i < len(arch) - 2:
                self.model.append(th.nn.BatchNorm1d(num_features=arch[i+1]))
                self.model.append(th.nn.ReLU())
    
    def forward(self, input:th.Tensor):
        
        embedded = self.encoder.forward(input)
        init_shape = embedded.shape

        #seq, batch, dim
        embedded = embedded.transpose(0,1)

        #batch seq dim
        embedded = embedded.reshape([embedded.shape[0], -1])

        #batch seq*dim
        internal_result = self.model.forward(embedded)
        internal_result = internal_result.reshape([init_shape[1], init_shape[0], init_shape[-1]])

        #batch seq dim
        internal_result = internal_result.transpose(0,1)

        #seq, batch, dim
        result = self.decoder(internal_result)
        return result

In [4]:
seq_len = 5
batch_size = 2
ntoken=100

tmlt = TMLP(seq_len=seq_len, arch=[5, 5, 5], ntoken=ntoken)

inpt = th.randint(0, ntoken, [seq_len, batch_size])
result = tmlt.forward(inpt)

In [5]:
tmlt

TMLP(
  (model): Sequential(
    (0): Linear(in_features=25, out_features=5, bias=True)
    (1): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=5, out_features=25, bias=True)
  )
  (encoder): Embedding(100, 5)
  (decoder): Linear(in_features=5, out_features=100, bias=True)
)

In [36]:
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

train_iter = WikiText2(split='train')
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

def data_process(raw_text_iter):
    """Converts raw text into a flat Tensor."""
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

# train_iter was "consumed" by the process of building the vocab,
# so we have to create it again
train_iter, val_iter, test_iter = WikiText2()
train_data = data_process(train_iter)
val_data = data_process(val_iter)
test_data = data_process(test_iter)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def batchify(data, bsz: int):
    """Divides the data into bsz separate sequences, removing extra elements
    that wouldn't cleanly fit.

    Args:
        data: Tensor, shape [N]
        bsz: int, batch size

    Returns:
        Tensor of shape [N // bsz, bsz]
    """
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    data = data.view(bsz, seq_len).t().contiguous()
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_data, batch_size).to('cuda')  # shape [seq_len, batch_size]
val_data = batchify(val_data, eval_batch_size).to('cuda')
test_data = batchify(test_data, eval_batch_size).to('cuda')

bptt = 35
def get_batch(source, i: int):
    """
    Args:
        source: Tensor, shape [full_seq_len, batch_size]
        i: int

    Returns:
        tuple (data, target), where data has shape [seq_len, batch_size] and
        target has shape [seq_len * batch_size]
    """
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len]
    return data, target

In [37]:
data, targets = get_batch(train_data, 0)

In [38]:
data.shape

torch.Size([35, 20])

In [39]:
targets.shape

torch.Size([35, 20])

In [20]:
batch_size = 2
seq_len = bptt
dim = 3
ntokens = len(vocab)  # size of vocabulary
inpt = th.ones([batch_size, dim, seq_len])
model = TMLP(seq_len=seq_len, arch=[180, 180, 180], ntoken=ntokens)

In [21]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(model=model)

12665142

In [22]:
model = model.to('cuda')

In [26]:
data, targets = get_batch(train_data,0)

In [28]:
data.shape

torch.Size([35, 20])

In [30]:
targets = targets.reshape([35, 20])

In [32]:
data[1]

tensor([ 3849,    12,   300,  6302,  3989,  1930, 10559,   451,     4,     7,
            2,  1511, 10115,   942,  2439,   572,     1,    47,    30,  1990],
       device='cuda:0')

In [33]:
targets[0]

tensor([ 3849,    12,   300,  6302,  3989,  1930, 10559,   451,     4,     7,
            2,  1511, 10115,   942,  2439,   572,     1,    47,    30,  1990],
       device='cuda:0')

In [42]:
criterion = nn.CrossEntropyLoss()
lr = 1e-3  # learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    
    num_batches = len(train_data) // bptt
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        seq_len = data.size(0)
        if seq_len != bptt:  # only on last batch
            break
        output = model.forward(data)
        loss = criterion(output[-1].view(-1, ntokens), targets[-1])

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            #lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

def evaluate(model: nn.Module, eval_data) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            seq_len = data.size(0)
            if seq_len != bptt:
                break
            output = model(data)
            output_flat = output[-1].view(-1, ntokens)
            total_loss += seq_len * criterion(output_flat, targets[-1]).item()
    return total_loss / (len(eval_data) - 1)

In [43]:
best_val_loss = float('inf')
epochs = 3
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model)
    val_loss = evaluate(model, val_data)
    val_ppl = math.exp(val_loss)
    elapsed = time.time() - epoch_start_time
    print('-' * 89)
    print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
          f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = copy.deepcopy(model)

    #scheduler.step()

| epoch   1 |   200/ 2928 batches | lr 0.00 | ms/batch 14.42 | loss  6.56 | ppl   704.40
| epoch   1 |   400/ 2928 batches | lr 0.00 | ms/batch 14.09 | loss  6.89 | ppl   986.79
| epoch   1 |   600/ 2928 batches | lr 0.00 | ms/batch 14.13 | loss  6.74 | ppl   848.07
| epoch   1 |   800/ 2928 batches | lr 0.00 | ms/batch 14.09 | loss  6.90 | ppl   994.60
| epoch   1 |  1000/ 2928 batches | lr 0.00 | ms/batch 14.16 | loss  6.84 | ppl   934.53
| epoch   1 |  1200/ 2928 batches | lr 0.00 | ms/batch 14.19 | loss  6.92 | ppl  1015.55
| epoch   1 |  1400/ 2928 batches | lr 0.00 | ms/batch 14.11 | loss  6.88 | ppl   970.41
| epoch   1 |  1600/ 2928 batches | lr 0.00 | ms/batch 14.14 | loss  6.93 | ppl  1023.11
| epoch   1 |  1800/ 2928 batches | lr 0.00 | ms/batch 14.06 | loss  6.82 | ppl   916.53
| epoch   1 |  2000/ 2928 batches | lr 0.00 | ms/batch 14.03 | loss  6.99 | ppl  1087.50
| epoch   1 |  2200/ 2928 batches | lr 0.00 | ms/batch 14.04 | loss  6.85 | ppl   947.08
| epoch   1 |  2400/ 

In [44]:
test_loss = evaluate(best_model, test_data)
test_ppl = math.exp(test_loss)
print('=' * 89)
print(f'| End of training | test loss {test_loss:5.2f} | '
      f'test ppl {test_ppl:8.2f}')
print('=' * 89)

| End of training | test loss  6.84 | test ppl   933.12
